# Uporaba PostgreSQL v Pythonu s knjižnico `psycopg`

## Nameščanje knjižnice `psycopg`

Namestitev izvedemo iz ukazne vrstice. Če nimamo administrativnih pravic na racunalniku, izvedemo:

```bash
python -m pip install --user psycopg[binary]
```


Če jih imamo:
```bash
python -m pip install psycopg[binary]
```

**Pozor:** za uporabo tega programa moramo imeti dostop do podatkovne baze PostgreSQL s pravico ustvarjanja tabel. Bazo PostgreSQL si lahko namestimo na svojem racunalniku in v datoteki `auth.py` nastavimo ustrezne podatke za dostop.

In [1]:
import psycopg
from auth import auth

Ustvarimo tabelo in jo napolnimo. Ker smo povezavo odprli s stavkom `with`, se bodo vse poizvedbe znotraj njega izvedle v eni transakciji - ob uspešnem izvajanju se bo ta potrdila, ob napaki pa preklicala. V vsakem primeru se bo povezava potem zaprla.

Tudi kurzor lahko odpremo s stavkom `with`, da se po koncu izvajanja zapre.

In [2]:
with psycopg.connect(**auth) as con:
    with con.cursor() as cur:   # "odzivnik" za pregledovanje poizvedbe
        cur.execute("DROP TABLE IF EXISTS Cars")
        cur.execute("CREATE TABLE Cars(Id INTEGER PRIMARY KEY, Name TEXT, Price INTEGER)")
        cur.execute("INSERT INTO Cars VALUES (1, 'Audi', 52642)")
        cur.execute("INSERT INTO Cars VALUES (2, 'Mercedes', 57127)")
        cur.execute("INSERT INTO Cars VALUES (3, 'Skoda', 9000)")
        cur.execute("INSERT INTO Cars VALUES (4, 'Volvo', 29000)")
        cur.execute("INSERT INTO Cars VALUES (5, 'Bentley', 350000)")
        cur.execute("INSERT INTO Cars VALUES (6, 'Citroen', 21000)")
        cur.execute("INSERT INTO Cars VALUES (7, 'Hummer', 41400)")
        cur.execute("INSERT INTO Cars VALUES (8, 'Volkswagen', 21600)")

Povezavo si lahko tudi shranimo in večkrat uporabimo. Parameter `autocommit=True` pri vzpostavljanju povezave določa, naj se vsaka poizvedba, ki je eksplicitno ne vključimo v transakcijo, takoj potrdi.

In [27]:
con = psycopg.connect(**auth, autocommit=True)

Primer: Katere vrstice so v tabeli?

In [20]:
cur1 = con.cursor()
cur1.execute("SELECT * from Cars")
print(cur1.fetchall())

[(1, 'Audi', 52642), (2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000), (5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600), (10, 'Ferrari', 192000), (11, 'Lamborghini', 400000), (9, 'Hyundai', 19200)]


Ko kurzor enkrat vrne podatke, se "izčrpa".

In [5]:
cur1.fetchall()

[]

Z `with con.transaction()` naredimo transakcijo - ob uspešnem koncu se potrdi, sicer bi se preklicala.

In [6]:
with con.transaction():
    cur1.execute("INSERT INTO Cars VALUES (9, 'Hyundai', 19200)")
    cur1.execute("INSERT INTO Cars VALUES (9, 'Ferrari', 192000)")

UniqueViolation: duplicate key value violates unique constraint "cars_pkey"
DETAIL:  Key (id)=(9) already exists.

In [8]:
with con.transaction():
    cur1.execute("INSERT INTO Cars VALUES (10, 'Ferrari', 192000)")
    cur1.execute("INSERT INTO Cars VALUES (11, 'Lamborghini', 400000)")

Transakcije lahko potrjujemo in preklicujemo tudi sami z metodama `con.commit()` oziroma `con.rollback()`.

In [10]:
con2 = psycopg.connect(**auth) # uporabimo novo povezavo brez autocommit

In [16]:
with con2.cursor() as cur2:
    cur2.execute("INSERT INTO Cars VALUES (9, 'Hyundai', 19200)")

In [17]:
with con2.cursor() as cur2:
    cur2.execute("SELECT * from Cars")
    print(cur2.fetchall())

[(1, 'Audi', 52642), (2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000), (5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600), (10, 'Ferrari', 192000), (11, 'Lamborghini', 400000), (9, 'Hyundai', 19200)]


In [14]:
con2.rollback()

In [19]:
con2.commit()

Primer: Poizvedba z iteracijo po 'odzivniku'.

In [21]:
with con.cursor() as cur:
    cur.execute("SELECT * from Cars")
    for podatek in cur:
        print(podatek)

(1, 'Audi', 52642)
(2, 'Mercedes', 57127)
(3, 'Skoda', 9000)
(4, 'Volvo', 29000)
(5, 'Bentley', 350000)
(6, 'Citroen', 21000)
(7, 'Hummer', 41400)
(8, 'Volkswagen', 21600)
(10, 'Ferrari', 192000)
(11, 'Lamborghini', 400000)
(9, 'Hyundai', 19200)


Primer interaktivne poizvedbe,  uporaba niza `%s`.

In [23]:
with con.cursor() as cur:
    kaj = input("Cena katerega vozila te zanima:")
    cur.execute("SELECT Name, Price FROM Cars WHERE Name LIKE %s", (kaj,))
    print(cur.fetchall())

Cena katerega vozila te zanima:'
[]


Primer interaktivne poizvedbe, uporaba `%(ključ)s`, preverjanje rezultata.

In [28]:
with con.cursor() as cur:
    kaj = input("Katero vozilo te zanima:").strip()
    cur.execute("SELECT Price FROM Cars WHERE Name LIKE %(ime)s", {"ime": kaj})
    rez = cur.fetchone()
    if rez is None:
        print("Ni takega vozila ({0}).".format(kaj))
    else:
        print("Cena vozila {0} je {1}.".format(kaj, rez[0]))

Katero vozilo te zanima:Ferrari
Cena vozila Ferrari je 192000.


Primer: večkratno vstavljanje.

In [29]:
cars = [
    (12, 'Audi2', 52643),
    (13, 'MercedesX', 57642),
    (14, 'Škoda', 9000),
    (15, 'Volvek', 29000),
    (16, 'Bentley7', 350000),
    (17, 'HummerU', 41400),
    (18, 'VolkswagenPassat', 21600)
]
print("Vstavljamo: ", cars)
with con.cursor() as cur:
    cur.executemany("INSERT INTO Cars VALUES (%s, %s, %s)", cars)
    cur.execute("SELECT * from Cars")
    print("V tabeli imamo:")
    print(cur.fetchall())

Vstavljamo:  [(12, 'Audi2', 52643), (13, 'MercedesX', 57642), (14, 'Škoda', 9000), (15, 'Volvek', 29000), (16, 'Bentley7', 350000), (17, 'HummerU', 41400), (18, 'VolkswagenPassat', 21600)]
V tabeli imamo:
[(1, 'Audi', 52642), (2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000), (5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600), (10, 'Ferrari', 192000), (11, 'Lamborghini', 400000), (9, 'Hyundai', 19200), (12, 'Audi2', 52643), (13, 'MercedesX', 57642), (14, 'Škoda', 9000), (15, 'Volvek', 29000), (16, 'Bentley7', 350000), (17, 'HummerU', 41400), (18, 'VolkswagenPassat', 21600)]


Ko povezav in kurzorjev več ne potrebujemo, jih zapremo.

In [30]:
cur1.close()
con.close()
con2.close()